In [ ]:
import json
import numpy as np
from openff.toolkit import Molecule
from pint import UnitRegistry
import matplotlib.pyplot as plt
from tabulate import tabulate
from visualization import show_oemol_struc
from cinnabar.stats import bootstrap_statistic
from matplotlib.backends.backend_pdf import PdfPages
from collections import defaultdict
from matplotlib import rcParams
from PIL import Image
import io

from openff.units import unit

from qcelemental.models.common_models import Model
from qcelemental.models import AtomicInput
from qcengine import compute

In [ ]:
def return_energy(qcmol, method, program, basis):
    result = compute(
    input_data=AtomicInput(molecule=qcmol, 
                           driver="gradient", 
                           model=Model(method=method, 
                                       basis=basis,
                                       )),
        program=program
    )

    return result

rcParams.update({"font.size": 12})
REF_SPEC = 'mp2/heavy-aug-cc-pv[tq]z + d:ccsd(t)/heavy-aug-cc-pvdz'
with open('../data/'+REF_SPEC.replace('/','_')+'_single_points_data.json', 'r') as file:
        ref_dict = json.load(file)
with open('../data/MP2_heavy-aug-cc-pVTZ_torsiondrive_data.json', 'r') as file:
    mp2_data = json.load(file)



In [ ]:
ureg = UnitRegistry()
hartree = 1 * ureg.hartree
HARTREE_TO_KCALMOL = hartree.to(ureg.kilocalorie/(ureg.avogadro_constant*ureg.mole)).magnitude
kj = 1 * ureg.kilojoule
kj_to_kcal = kj.to(ureg.kilocalorie)

In [ ]:
program = 'psi4'
keywords_list = [
    "b97-d3bj/def2-tzvp",
    "m05-2x-d3/dzvp",
    "m06-2x-d3/dzvp",
    "m08-hx-d3/dzvp",
    # "wb97x-d3bj/dzvp", commented out since the dispersion energies are not handled properly in current qcf 
    # version 
    # for this functional
    "wb97m-d3bj/dzvp",
    "wb97m-v/dzvp",
    "pw6b95-d3bj/dzvp",
    "pw6b95-d3/dzvp",
    "mp2/aug-cc-pvtz",
    "mp2/heavy-aug-cc-pvtz",
    "dsd-blyp-d3bj/heavy-aug-cc-pvtz",
    "b3lyp-d3bj/dzvp",
    "b3lyp-nl/dzvp",
    "b3lyp-d3bj/def2-sv(p)",
    "b3lyp-d3bj/def2-svp",
    "b3lyp-d3bj/def2-tzvp",
    "b3lyp-d3bj/def2-tzvpd",
    "b3lyp-d3bj/def2-tzvpp",
    "b3lyp-d3bj/def2-tzvppd",
    "b3lyp-d3bj/def2-qzvp",
    "b3lyp-d3bj/6-31g*",
    "b3lyp-d3bj/6-31+g**",
    "b3lyp-d3bj/6-311+g**",
]

In [ ]:
walltimes = defaultdict(dict)
i = 0
for key in keywords_list:
    method = keywords_list.split('/')[0]
    basis = keywords_list.split('/')[1]
    mapped_smiles = mp2_data[str(i)]['metadata']['mapped_smiles']
    offmol = Molecule.from_mapped_smiles(mapped_smiles, allow_undefined_stereo=True)
    method_walltimes = []
    
    for j in range(10):
        positions = np.array(mp2_data[str(i)]['final_geometries'][j]) * 0.529177
        offmol._conformers = [positions * unit.angstrom]
        qcmol = offmol.to_qcschema()
        result = return_energy(qcmol, method=method, program=program, basis=basis)
        method_walltimes.append(result.provenance.)
    
    mean_time = np.mean(method_walltimes)
    stdev_time = np.stdev(method_walltimes)
    walltimes[key] = {'Mean': mean_time, 'Stdev': stdev_time}    

In [ ]:
with open('../data/timings_data.json', 'w') as outfile:
    json.dump(walltimes, outfile)       